# Gridded datasets

ERDDAP has a seperate mechanism for gridded data, like satellite and model/reanalysis products. This is called griddap. 

In [ ]:
from erddapy import ERDDAP
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs

### Specify server and protocol

Here we use a hi-res SST product from JPL via the coastwatch server https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplMURSST41.html

In [ ]:
e = ERDDAP(
    server="https://coastwatch.pfeg.noaa.gov/erddap",
    protocol="griddap",
)

e.dataset_id='jplMURSST41'

e.griddap_initialize() # This sends a background request to find the dataset's variables and extent
print(f"variables in this dataset:\n\n{e.variables}")


In [ ]:
e.constraints['latitude_step'] = 10
e.constraints['longitude_step'] = 10
e.variables = ['analysed_sst']

In [ ]:
ds = e.to_xarray()

In [ ]:
ds

In [ ]:
ds.analysed_sst.plot(figsize=(12, 8));

Gridded datasets can be very large. By default, erddapy returns only the most recent time slice of a dataset. This behaviour can be changed by adjusting the constraints, much like with tabledap. The mechanism for subsetting griddap data is a little more complicated, as the data are multi-dimensional.

Let's zoom in on the Gulf Stream and take a few time slices. To achieve this, we modify the constraints. We will tighten the lon/lat to a window of interest and request a wider timespan of data, taking every 30th day over the last few years

In [ ]:
e.constraints = {'time>=': '2023-01-01',
 'time<=': '2023-05-01',
 'time_step': 30,
 'latitude>=': 25,
 'latitude<=': 45,
 'latitude_step': 1,
 'longitude>=': -90,
 'longitude<=': -30,
 'longitude_step': 1}

e.variables=['analysed_sst']

ds_gulf = e.to_xarray()

In [ ]:
xr.plot.contourf(ds_gulf.sel(time="'2023-01-01", method="nearest").analysed_sst, vmin=10, vmax=30, levels=100, figsize=(14, 8));

In [ ]:
xr.plot.contourf(ds_gulf.sel(time="'2023-06-01", method="nearest").analysed_sst, vmin=10, vmax=30, levels=100, figsize=(14, 8));

----------------------------

### References

coastwatch ERDDAP https://coastwatch.pfeg.noaa.gov/erddap/index.html

For more information on how erddapy deals with griddap, see the [erddapy documentation example notebook](https://ioos.github.io/erddapy/01a-griddap-output.html)

### extra example: low-res topography

In [ ]:
pacioos = ERDDAP(
    server="https://pae-paha.pacioos.hawaii.edu/erddap",
    protocol="griddap",
)
pacioos.dataset_id = "etopo5_lon180"
pacioos.griddap_initialize()

pacioos.constraints = {
 'latitude>=': 20,
 'latitude<=': 60,
 'latitude_step': 1,
 'longitude>=': -90,
 'longitude<=': -30,
 'longitude_step': 1}
topo = pacioos.to_xarray()


In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()}, figsize=(10, 6))
topo["ROSE"].plot(ax=ax)
ax.coastlines();